In [1]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import Dash, dcc, html, Input, Output, callback
from io import StringIO
from base64 import b64decode

In [8]:
def plot_cum_count(df, column='Date Added'):
    date_added = pd.to_datetime(df[column]).sort_values(ascending=True)
    
    fig = go.Figure()
    for year, g in date_added.groupby(date_added.dt.year, sort=False):
        fig.add_trace(
            go.Scatter(
                x=g.apply(lambda x: x.replace(year=2024)).unique(), 
                y=g.value_counts(sort=False).cumsum(), 
                name=year,
                opacity=0.8,
                mode='lines',
            )
        )
    fig.update_layout(
        yaxis_title='Cumulative count',
        legend_title_text='Year',
        xaxis=dict(
            tickformatstops=[
                dict(dtickrange=[None, 'M1'], value='%e %b'),
                dict(dtickrange=['M1', None], value='%b')
            ],
            ticklabelmode='period', title='Date'
        ),
        title='Books by year', title_x=0.5
    )
    return fig

In [9]:
app = Dash()
app.layout = html.Div([
    dcc.Upload(
        id='upload',
        accept='.csv',
        children=[html.Button('Upload File')]
    ), html.Br(),
    dcc.Dropdown([''], '', id='feature_dropdown'),
    dcc.Graph(id='cum_count_graph'),
])

@callback(
    Output('cum_count_graph', 'figure'),
    Output('feature_dropdown', 'options'),
    Input('upload', 'contents'),
    Input('feature_dropdown', 'value'),
    prevent_initial_call=True,
)
def update_output(file_contents, feature_dropdown):
    df = pd.read_csv(
        StringIO(
            b64decode(file_contents.split(',')[1]).decode("utf-8")
        )
    )
    cum_count_graph = plot_cum_count(
        df, column='Date Added'
    )
    return cum_count_graph, df.columns

In [10]:
app.run(jupyter_mode='inline')